In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

In [31]:
path = '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv'
df1 = pd.read_csv(path).iloc[:,1:]
df1.drop(columns = ['Veteran flag','Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)


path = '/home/daisy/FDA_Dataset/inpatient_all_final_2.csv'
df2 = pd.read_csv(path).iloc[:,1:]
df2.drop(columns = ['Veteran flag','Event date','Marital status', 'Marital status encoded',
                    'State','Ruca category'], inplace=True)

In [12]:
df1.shape

(84536, 81)

### Train test split


In [26]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84536 entries, 0 to 84535
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Internalpatientid             84536 non-null  int64  
 1   num_stays                     84536 non-null  int64  
 2   stay_length                   84536 non-null  float64
 3   num_unique_units              84536 non-null  int64  
 4   num_transfers                 84536 non-null  int64  
 5   num_cvd_readmission           84536 non-null  int64  
 6   Readmission                   84536 non-null  int64  
 7   Died                          84536 non-null  int64  
 8   AO                            84536 non-null  int64  
 9   CVD                           84536 non-null  int64  
 10  unique_admitting_specialty    84536 non-null  int64  
 11  unique_discharging_specialty  84536 non-null  int64  
 12  DOMICILIARY                   84536 non-null  int64  
 13  M

In [36]:
X_admission = df1.drop(columns = ['Readmission','Died'])
Y_admission = df1[['Readmission']]

X_mortality = df1.drop(columns = ['Died'])
Y_mortality = df1[['Died']]

In [37]:
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_admission, Y_admission, test_size=0.20, random_state=42)
X_train_mor, X_test_mor, y_train_mor, y_test_mor = train_test_split(X_mortality, Y_mortality, test_size=0.20, random_state=42)


#### Filling missing values

In [38]:
df1.columns[df1.isna().any()].tolist()

['total_procedure',
 'num_surgery_pro',
 'num_immunization',
 'Num med per admission mean',
 'Num med per admission min',
 'Num med per admission max',
 'Total medications',
 'mean age at specailty',
 'period mean',
 'period std',
 'specialty medical count',
 'specialty support count',
 'specialty count',
 'Age 20-40 hypotension',
 'Age 40-60 hypotension',
 'Age 60-80 hypotension',
 'Age 80-100 hypotension',
 'Age 100-120 hypotension',
 'Age 20-40 hypertension',
 'Age 40-60 hypertension',
 'Age 60-80 hypertension',
 'Age 80-100 hypertension',
 'Age 100-120 hypertension',
 'Age 20-40 healthy',
 'Age 40-60 healthy',
 'Age 60-80 healthy',
 'Age 80-100 healthy',
 'Age 100-120 healthy',
 'lab_count',
 'lab_freq',
 'lab_age_mean',
 'lab_age_std']

In [39]:
# Filling continuous features with mean values
numeric_cols = df1.columns[df1.isna().any()].tolist()

X_train_ad[numeric_cols] = X_train_ad[numeric_cols].fillna(X_train_ad[numeric_cols].mean())
X_train_mor[numeric_cols] =  X_train_mor[numeric_cols].fillna(X_train_mor[numeric_cols].mean())

X_test_ad[numeric_cols] = X_test_ad[numeric_cols].fillna(X_test_ad[numeric_cols].mean())
X_test_mor[numeric_cols] = X_test_mor[numeric_cols].fillna(X_test_mor[numeric_cols].mean())

In [40]:
X_train_ad.isna().any().sum()

0

#### Logistic Regression

In [41]:
lr =LogisticRegression()
# Training
lr.fit(X_train_ad, y_train_ad)

# Prediction
lr_prediction = lr.predict(X_test_ad)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [42]:
print(classification_report(y_test_ad, lr_prediction, target_names= ['Not Readmitted', 'Readmitted']))


                precision    recall  f1-score   support

Not Readmitted       0.87      0.85      0.86      3316
    Readmitted       0.96      0.97      0.97     13592

      accuracy                           0.95     16908
     macro avg       0.92      0.91      0.91     16908
  weighted avg       0.94      0.95      0.94     16908



In [43]:
lr = LogisticRegression()
# Training
lr.fit(X_train_mor, y_train_mor)

# Prediction
lr_prediction = lr.predict(X_test_mor)

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
print(classification_report(y_test_mor, lr_prediction, target_names= ['Not Died', 'Died']))


              precision    recall  f1-score   support

    Not Died       0.83      0.99      0.90     13744
        Died       0.65      0.11      0.19      3164

    accuracy                           0.82     16908
   macro avg       0.74      0.55      0.55     16908
weighted avg       0.79      0.82      0.77     16908



In [45]:
import statsmodels.api as sm
logit_model=sm.Logit(y_train_mor,X_train_mor)
result=logit_model.fit()
print(result.summary2()) 

Optimization terminated successfully.
         Current function value: 0.371450
         Iterations 11
                                       Results: Logit
Model:                       Logit                     Pseudo R-squared:          0.224     
Dependent Variable:          Died                      AIC:                       50378.8745
Date:                        2023-07-18 15:45          BIC:                       51008.2771
No. Observations:            67628                     Log-Likelihood:            -25120.   
Df Model:                    68                        LL-Null:                   -32361.   
Df Residuals:                67559                     LLR p-value:               0.0000    
Converged:                   1.0000                    Scale:                     1.0000    
No. Iterations:              11.0000                                                        
--------------------------------------------------------------------------------------------
      

In [46]:
classifiers = {
    "LogisiticRegression": LogisticRegression(),
    "Linear Discriminant Analysis":  LinearDiscriminantAnalysis(solver='lsqr')
}

In [48]:
from sklearn.model_selection import cross_val_score


for key, classifier in classifiers.items():
    classifier.fit(X_train_mor, y_train_mor)
    training_score = cross_val_score(classifier, X_train_mor, y_train_mor, cv=5)
    print(classifier.__class__.__name__,"Classifier has a training score of", round(training_score.mean(), 2) * 100, "% accuracy score")

/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHE

LogisticRegression Classifier has a training score of 82.0 % accuracy score


/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hassan/.conda/envs/mla/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d arr

LinearDiscriminantAnalysis Classifier has a training score of 83.0 % accuracy score


In [6]:
from sklearn.ensemble import VotingClassifier
import pickle
import pandas as pd
import numpy as np
from importlib import reload
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.utils.validation import column_or_1d
import pickle
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import argparse
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from transformation import *    
from grid_search_cv import *
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.datasets import dump_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics   #Additional scklearn functions
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier

In [51]:
LR_best = pickle.load(open('/home/daisy/FDA/models/LogisticRegression_readmission_2.sav','rb'))
RF_best = pickle.load(open('/home/daisy/FDA/models/RandomForest_readmission_2.sav','rb'))
DT_best = pickle.load(open('/home/daisy/FDA/models/DecisionTree_readmission_2.sav','rb'))
XGB_best = pickle.load(open('/home/daisy/FDA/models/XGBoost_readmission_2.sav','rb'))
LDA_best = pickle.load(open('/home/daisy/FDA/models/LinearDiscriminant_readmission_2.sav','rb'))


In [97]:
dict_data = {
    'readmission': '/home/daisy/FDA_Dataset/inpatient_all_final_1.csv', 
    'readmission_cvd': '/home/daisy/FDA_Dataset/inpatient_CVD_final_1.csv',
    "mortality": '/home/daisy/FDA_Dataset/final_allcause_mortality_train_1.csv',
    "mortality_cvd": '/home/daisy/FDA_Dataset/final_cvd_mortality_train_1.csv'
}

def prepare_dataset(target):
    # Import Data
    path =  dict_data[target]
    data = pd.read_csv(path).iloc[:,1:]
   
    data_X = data[['Age 100-120 hypertension',
 'Age 20-40 healthy',
 'Age 40-60',
 'Age 40-60 healthy',
 'Age 40-60 hypertension',
 'Age 60-80 healthy',
 'Age 60-80 hypertension',
 'Age 60-80 hypotension',
 'Age 80-100 healthy',
 'Age 80-100 hypertension',
 'Age 80-100 hypotension',
 'CVD',
 'DOMICILIARY',
 'Ethnicity_0',
 'Height',
 'MEDICINE',
 'Num med per admission mean',
 'Others_Specialty',
 'Pulse oximetry max',
 'Pulse oximetry mean',
 'Pulse oximetry min',
 'Pulse oximetry std',
 'Races_0',
 'Races_2',
 'SURGERY',
 'Total medications',
 'Weight',
 'age_std',
 'lab_age_mean',
 'lab_age_std',
 'lab_count',
 'lab_freq',
 'mean age at specailty',
 'num_stays',
 'period mean',
 'period std',
 'specialty count',
 'specialty medical count',
 'specialty support count',
 'stay_length',
 'stay_max',
 'stay_mean',
 'stay_min',
 'stay_std',
 'total_procedure','Ethnicity', 'Gender', 'Races', 'Ethnicity_1', 'Ethnicity_2', 'Races_1', 'Races_3', 'Ruca category encoded']]
    if target == "readmission":
        #X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
        X = data_X
        y = column_or_1d(data[['readmission within 300 days']])
    elif target == "readmission_cvd":
        X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
        y = column_or_1d(data[['CVD_readmission']])
    elif target == "mortality":
        X = data.drop(columns = ['Internalpatientid', 'died_within_125days'])
        y = column_or_1d(data[['died_within_125days']])
    else:
        X = data.drop(columns = ['Internalpatientid','died_by_cvd','Age at death'])
        y = column_or_1d(data[['died_by_cvd']])
        

    # # Split Train and Test (?? 似乎不用)
    # X_train_ad1, X_test_ad1, y_train_ad1, y_test_ad1 = train_test_split(X_admission1, Y_admission1, test_size=0.20, random_state=42)
    # Transform Data
    transform_steps = [("ImputeNumeric", ImputeNumeric()),
                ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis()),
                ('StandardizeStandardScaler', Standardize(RobustScaler()))]
    transform_pipeline = Pipeline(transform_steps)

    X = transform_pipeline.transform(X)
    unique, counts = np.unique(y, return_counts=True)
    print(unique, counts)

    # Balance the dataset
    sme = SMOTEENN(random_state=42)
    X, y = sme.fit_resample(X, y)
    unique, counts = np.unique(y, return_counts=True)
    print(unique, counts)
    
    return X,y


In [98]:
X,y = prepare_dataset("readmission")
eclf1 = VotingClassifier(estimators=[('rf', RF_best),('lr', LR_best),
        ('dt', DT_best)], voting='hard',weights=[2,1,1], n_jobs=3)
eclf1 = eclf1.fit(X, y)

/home/daisy/FDA/src/models/transformation.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[colname].fillna((X[colname].mean()), inplace = True)
/home/daisy/FDA/src/models/transformation.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[colname+'_log'] = np.log1p(X[colname])
/home/daisy/FDA/src/models/transformation.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

[0 1] [40819 43717]
[0 1] [14552 11950]


In [ ]:
eclf2 = VotingClassifier(estimators=[('rf', RF_best),('lr', LR_best),
        ('dt', DT_best)], voting='hard',weights=[2,1,1], n_jobs=3)
eclf2 = eclf2.fit(X, y)

In [99]:
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import numpy as np
#np.random.seed(401)


def get_AUPRC(y_test, y_pred):
    average_precision_score(y_test, y_pred)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
    # Use AUC function to calculate the area under the curve of precision recall curve
    auc_precision_recall = auc(recall, precision)
    return auc_precision_recall

def get_AUROC(y_test, y_pred):
    auc_roc_score = roc_auc_score(y_test, y_pred)
    return auc_roc_score

def get_Accurarcy(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

def get_SensitSpecific(y_test, y_pred):
    sensitivity = recall_score(y_test, y_pred,average = 'binary') #TP / (TP + FN)

    #As it was mentioned in the other answers, specificity is the recall of the negative class
    specificity = recall_score(y_test, y_pred, pos_label=0) # TN / (TN + FP) 
    return sensitivity + specificity

def get_Sensitivity(y_test, y_pred):
    sensitivity = recall_score(y_test, y_pred,average = 'binary') #TP / (TP + FN)
    return sensitivity

def get_Specificity(y_test, y_pred):
    specificity = recall_score(y_test, y_pred, average = 'binary', pos_label=0) # TN / (TN + FP) 
    return specificity

def get_Precision(y_test, y_pred):
    precision = precision_score(y_test, y_pred, average='binary')
    return precision

def get_F1score(y_test, y_pred):
    score = f1_score(y_test, y_pred, average='binary')
    return score

def get_NPC(y_test, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    npc =  tn / (tn + fn)
    return npc

def get_PositiveLR(y_test, y_pred):
    # sensitivity / (1 - specificity)
    positive_lr = get_Sensitivity(y_test, y_pred) / (1 - get_Specificity(y_test, y_pred))
    return positive_lr

def get_NegativeLR(y_test, y_pred):
    # (1 - sensitivity) / specificity
    negative_lr = (1 - get_Sensitivity(y_test, y_pred)) / get_Specificity(y_test, y_pred)
    return negative_lr

In [100]:
dict_target_info = {
    'mortality': ['/home/daisy/FDA_Dataset/final_allcause_mortality_test_1.csv','/home/vivi/FDA/models/RandomForest_mortality.sav'],
    'mortality_cvd':['/home/daisy/FDA_Dataset/final_cvd_mortality_test_1.csv', '/home/vivi/FDA/models/RandomForest_mortality_cvd.sav'],
    'readmission': ['/home/daisy/FDA_Dataset/inpatient_all_final_test_1.csv', "/home/vivi/FDA/models/LinearDiscriminant_readmission.sav"],
    'readmission_cvd': ['/home/daisy/FDA_Dataset/inpatient_CVD_final_test_1.csv', '/home/vivi/FDA/models/DecisionTree_readmission_cvd.sav']
}


def prepare_dataset(target,feature_names):
    # Import Data
    path =  dict_target_info[target][0]
    data = pd.read_csv(path).iloc[:,1:]
    
    data_X = data[['Age 100-120 hypertension',
 'Age 20-40 healthy',
 'Age 40-60',
 'Age 40-60 healthy',
 'Age 40-60 hypertension',
 'Age 60-80 healthy',
 'Age 60-80 hypertension',
 'Age 60-80 hypotension',
 'Age 80-100 healthy',
 'Age 80-100 hypertension',
 'Age 80-100 hypotension',
 'CVD',
 'DOMICILIARY',
 'Ethnicity_0',
 'Height',
 'MEDICINE',
 'Num med per admission mean',
 'Others_Specialty',
 'Pulse oximetry max',
 'Pulse oximetry mean',
 'Pulse oximetry min',
 'Pulse oximetry std',
 'Races_0',
 'Races_2',
 'SURGERY',
 'Total medications',
 'Weight',
 'age_std',
 'lab_age_mean',
 'lab_age_std',
 'lab_count',
 'lab_freq',
 'mean age at specailty',
 'num_stays',
 'period mean',
 'period std',
 'specialty count',
 'specialty medical count',
 'specialty support count',
 'stay_length',
 'stay_max',
 'stay_mean',
 'stay_min',
 'stay_std',
 'total_procedure','Ethnicity', 'Gender', 'Races', 'Ethnicity_1', 'Ethnicity_2', 'Races_1', 'Races_3', 'Ruca category encoded']]
    if target == "readmission":
        #X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
        X = data_X
        y = column_or_1d(data[['readmission within 300 days']])
    elif target == "readmission_cvd":
        X = data.drop(columns = ['Internalpatientid', 'CVD_readmission', 'readmission within 300 days'])
        y = column_or_1d(data[['CVD_readmission']])
    elif target == "mortality":
        X = data.drop(columns = ['Internalpatientid', 'died_within_125days'])
        y = column_or_1d(data[['died_within_125days']])
    else:
        print(target)
        X = data.drop(columns = ['Internalpatientid','died_by_cvd','Age at death'])
        y = column_or_1d(data[['died_by_cvd']])
    
    # Transform Data
    transform_steps = [("ImputeNumeric", ImputeNumeric()),
                ('RemoveSkewnessKurtosis', RemoveSkewnessKurtosis(feature_names)),
                ('StandardizeStandardScaler', Standardize(RobustScaler()))]
    transform_pipeline = Pipeline(transform_steps)

    X = transform_pipeline.transform(X)

    X = X[feature_names]
    return X,y

def get_patientId(target):
    path = dict_target_info[target][0]
    data = pd.read_csv(path).iloc[:,1:]
    patientId = pd.DataFrame(data['Internalpatientid'])
    return patientId

def make_prediction(X,target,clf):
    predict_label = clf.predict(X)
    #predict_contin = [pair[1] for pair in clf.predict_proba(X)] #for soft
    # return predict_label, predict_contin #for soft
    return predict_label

def calculate_score(y, predict_label):
    scores = [
        get_AUPRC(y, predict_label),
        get_AUROC(y, predict_label),
        get_Accurarcy(y, predict_label),
        get_SensitSpecific(y, predict_label),
        get_Sensitivity(y, predict_label),
        get_Specificity(y, predict_label),
        get_Precision(y, predict_label),
        get_NPC(y, predict_label),
        get_PositiveLR(y, predict_label),
        get_NegativeLR(y, predict_label),
        get_F1score(y, predict_label)
    ]
    return scores

def make_df():
    statistics_metrics1 = pd.DataFrame(['Area under the precision recall curve (AUPRC)',
                                       'Area under the Receiver Operating Characteristic (AUROC)',
                                       'Overall Accuracy',
                                       'Sum of Sensitivity and Specificity',
                                       'Sensitivity',
                                       'Specificity',
                                       'Precision',
                                       'Negative Predictive Value',
                                       'Positive Likelihood Ratio',
                                       'Negative Likelihood Ratio',
                                       'F1 score'], columns=['statistics_metrics'])
    target = 'readmission'
    X_test1, y_test1= prepare_dataset(target, eclf1.feature_names_in_)
    # predict_label, predict_contin = make_prediction(X,target,eclf1) # for soft
    predict_label = make_prediction(X_test1,target,eclf1)
    scores1 = calculate_score(y_test1, predict_label)
    statistics_metrics1[target] = scores1
    return statistics_metrics1

In [101]:
make_df()


/home/daisy/FDA/src/models/transformation.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[colname].fillna((X[colname].mean()), inplace = True)
/home/daisy/FDA/src/models/transformation.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[colname+'_log'] = np.log1p(X[colname])
/home/daisy/FDA/src/models/transformation.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,statistics_metrics,readmission
0,Area under the precision recall curve (AUPRC),0.794082
1,Area under the Receiver Operating Characterist...,0.715626
2,Overall Accuracy,0.715614
3,Sum of Sensitivity and Specificity,1.431252
4,Sensitivity,0.715198
5,Specificity,0.716054
6,Precision,0.726707
7,Negative Predictive Value,0.704279
8,Positive Likelihood Ratio,2.518782
9,Negative Likelihood Ratio,0.397738
